# This is to convert from nmea to Plane Rectangular. Coordinate System by GSI(Geospatial Information Authority of Japan) HP.

# 緯度経度(nmea)から平面直角座標へ国土地理院のHPで変換したい場合

https://vldb.gsi.go.jp/sokuchi/surveycalc/surveycalc/bl2xyf.html

when calculate again,

if you want

%reset

In [1]:
#if you want
#check current directory
%pwd

'C:\\Users\\survey\\Dropbox\\My PC (SURVEY-C20)\\Documents\\GitHub\\GNSS'

In [1]:
#define working directroy
# %cd C:\\Users\\survey\\Desktop\\nmea20210705
%cd C:\Users\survey\Dropbox\My PC (SURVEY-C20)\Documents\GitHub\GNSS

C:\Users\survey\Dropbox\My PC (SURVEY-C20)\Documents\GitHub\GNSS


In [30]:
#check files
%ls

 ドライブ C のボリューム ラベルがありません。
 ボリューム シリアル番号は 3C86-6F6B です

 C:\Users\survey\Dropbox\My PC (SURVEY-C20)\Documents\GitHub\GNSS のディレクトリ

2021/07/09  20:08    <DIR>          .
2021/07/09  19:44    <DIR>          ..
2021/07/09  19:17               195 .gitignore
2021/07/09  20:06    <DIR>          .idea
2021/07/09  20:08            13,944 gnss.ipynb
2021/07/09  20:00                46 README.md
2021/07/09  18:49            60,812 VRS直_2021-07-05-12-48-07.rtk
               4 個のファイル              74,997 バイト
               3 個のディレクトリ  1,048,534,614,016 バイトの空き領域


In [2]:
import pandas as pd
import numpy as np
import csv

# 1. まずは，入力ファイルを設定します.

In [3]:
inp_file="2021-07-05_10.nmea"
# # inp_file="VRS直_2021-07-05-12-48-07.rtk"
# inp_file="VRS直_2021-06-27-11-45-43.rtk"

# base_nameを取得しておきます．
# 出力ファイル名を自動で作成するため

In [4]:
type(inp_file)
base_name=inp_file.split('.',1)[0]
base_name
# inp_file="VRS直_2021-07-05-12-48-07_get.rtk"

'2021-07-05_10'

In [5]:
# make output filename from input file automatically

# out_file="2021-07-05_10_GNRMC_out_test.csv"
# out_file="VRS直_2021-07-05-12-48-07_get.rtk"
# out_file=base_name+"_get.rtk"
out_file=base_name+"_GSI.in"
out_file
# out_file="VRS直_2021-07-05-12-48-07_get.rtk"

'2021-07-05_10_GSI.in'

## nmeaはUTF-8
## Trimbleはshift_jisなはず

In [6]:
enc_sty="UTF-8"

# enc_sty="shift_jis"

In [25]:
# with open("2021-07-05_10.nmea", "r", encoding="UTF-8", errors="", newline="" ) as f:
# with open(inp_file, "r", encoding="UTF-8", errors="", newline="" ) as f:
with open(inp_file, "r", encoding=enc_sty, errors="", newline="" ) as f:
    lst = csv.reader(f, delimiter=",")
    df = pd.DataFrame(lst).rename(columns={0:'ID',1:'time',2:'de',3:'lat',5:'lon',7:'knot',9:'date',12:'mode'})
    # df = pd.DataFrame(lst)

get_id='GNRMC'

In [26]:
df_gnrmc=df.query('ID in ["$GNRMC"]')
df_gnrmc

,ID,time,de,lat,4,lon,6,knot,8,date,...,11,mode,13,14,15,16,17,18,19,20
11,$GNRMC,005959.88,A,3511.0128890,N,13707.1113739,E,0.071,,050721,...,,R,V*01,None,None,None,None,None,None,None
27,$GNRMC,010000.00,A,3511.0128878,N,13707.1113731,E,0.048,,050721,...,,R,V*04,None,None,None,None,None,None,None
43,$GNRMC,010000.13,A,3511.0128907,N,13707.1113732,E,0.048,,050721,...,,R,V*0C,None,None,None,None,None,None,None
59,$GNRMC,010000.25,A,3511.0128898,N,13707.1113735,E,0.043,,050721,...,,R,V*02,None,None,None,None,None,None,None
75,$GNRMC,010000.38,A,3511.0128892,N,13707.1113735,E,0.030,,050721,...,,R,V*00,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68130,$GNRMC,010834.38,A,3511.0129030,N,13707.1113436,E,0.015,,050721,...,,R,V*09,None,None,None,None,None,None,None
68149,$GNRMC,010834.50,A,3511.0129025,N,13707.1113439,E,0.024,,050721,...,,R,V*0E,None,None,None,None,None,None,None
68168,$GNRMC,010834.63,A,3511.0129041,N,13707.1113448,E,0.015,,050721,...,,R,V*08,None,None,None,None,None,None,None
68187,$GNRMC,010834.75,A,3511.0129040,N,13707.1113448,E,0.025,,050721,...,,R,V*0D,None,None,None,None,None,None,None


In [28]:
df_gnrmc_ll=df_gnrmc.filter(items=['lat','lon'])
df_gnrmc_ll

,lat,lon
11,3511.0128890,13707.1113739
27,3511.0128878,13707.1113731
43,3511.0128907,13707.1113732
59,3511.0128898,13707.1113735
75,3511.0128892,13707.1113735
...,...,...
68130,3511.0129030,13707.1113436
68149,3511.0129025,13707.1113439
68168,3511.0129041,13707.1113448
68187,3511.0129040,13707.1113448


In [29]:
df_gnrmc_ll = df_gnrmc_ll.reset_index(drop=True)
df_gnrmc_ll

,lat,lon
0,3511.0128890,13707.1113739
1,3511.0128878,13707.1113731
2,3511.0128907,13707.1113732
3,3511.0128898,13707.1113735
4,3511.0128892,13707.1113735
...,...,...
3948,3511.0129030,13707.1113436
3949,3511.0129025,13707.1113439
3950,3511.0129041,13707.1113448
3951,3511.0129040,13707.1113448


# ここでnumpy化します

In [46]:
df_gnrmc_ll_np=df_gnrmc_ll.values
df_gnrmc_ll_np

array([['3511.0128890', '13707.1113739'],
       ['3511.0128878', '13707.1113731'],
       ['3511.0128907', '13707.1113732'],
       ...,
       ['3511.0129041', '13707.1113448'],
       ['3511.0129040', '13707.1113448'],
       ['3511.0129026', '13707.1113445']], dtype=object)

In [47]:
type(df_gnrmc_ll_np)
# df_gnrmc_ls

numpy.ndarray

In [44]:
a=10
a
type(a)

int

In [48]:
df_gnrmc_ll_np + 10

TypeError: can only concatenate str (not "int") to str

In [34]:
lat_int=df_gnrmc_ls[:,1])

TypeError: only size-1 arrays can be converted to Python scalars

In [ ]:
# df.query('cols in ["GG1","GG2"]')
# df.query('ID in ["GG1","GG2"]').to_csv(out_file,header=False, index=False)
df.query('ID in ["$GNRMC"]').to_csv(out_file,header=False, index=False)
# df[df[0].str.contains(id)].to_csv(out_file,header=False, index=False)

FIN
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# 灯台下暗し
# 欲しい列だけ指定
df_gnrmc=df_gnrmc.drop(df_gnrmc.columns[[0,1,2,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]], axis=1)
# concatenated_df=concatenated_df.drop(concatenated_df.columns[[0,1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,20,21,22,23,24,25,26,27,28,29,30,31]], axis=1)


In [ ]:
# concatenate horizontally

# id='GNRMC'
id="GG1|GG2"


df

# oi=df.iloc[0]
# print(oi)
oi2=df.iloc[:,1]
print(oi2)

# print(df[0].str.contains('$GNRMC'))
# df[0].str.contains('GNRMC')
df[0].str.contains(id)



# df[df[0].str.contains('GNRMC')].to_csv("2021-07-05_10_GNRMC.csv",header=False, index=False)
df[df[0].str.contains(id)].to_csv(out_file,header=False, index=False)

##################################################################
#################################
FIN

pd.options.display.max_rows = None
pd.options.display.max_columns = None

# df = pd.read_csv('2021-07-04_16.csv', index_col=0)
# df = pd.read_csv('2021-07-04_16.csv')
df = pd.read_csv('2021-07-04_16.csv', header=None)
print(df)

csvfile = '2021-07-05_07.nmea'
f = open(csvfile, "r")
reader = csv.reader(f)
# header = next(reader)

for row in reader:
        print(row)

f.close()


# df[0].isin([GNRMC])


df.to_csv("GNRMMC_1.csv")

type(kore)

print(df[df[0].str.contains('$GNRMC')])

df.iloc[5]

df.loc["$GNRMC"]


